## 1.3 신경망의 학습
학습되지 않은 신경망은 '좋은 추론'을 해낼 수 없다.  
그래서 학습을 먼저 수행하고, 그 학습도니 매개변수를 이용해 추론을 수행하는 것이 일반적이다.  
추론이란 앞 절에서 본 것 같은 다중 클래스 분류등 문제에 답을 구하는 작업이다.  
신경망의 학습은 최적의 매개 변수 값을 찾는 작업입니다.

## 1.3.1 손실 함수
신경망 학습에는 학습이 얼마나 잘 되고있는지를 알기 위한 '척도'가 필요로 한다.  
일반적으로 학습 단계의 특정 시점에서 신경망의 성능을 나타내는 척도로 손실(Loss)을 사용한다.  
손실은 학습 데이터(학습시 주어진 정답 데이터)와 신경망이 예측한 결과를 비교하여 예측이 얼마나 나쁜가를 산출한 단일 값(스칼라)이다.  

신경망의 손실은 손실 함수(Loss function)를 사용해 구한다.  
다중 클래스 분류(multi-class classification) 신경망에서는 손실 함수로 흔히 교차 엔트로피 오차(Cross Entropy Error)를 사용한다.  
교차 엔트로피 오차는 신경망이 출력하는 각 클래스의 확률과 정답 레이블을 이용해 구할 수 있다.  

#### SoftMax function
소프트맥스 함수의 출력의 각 원소는 0.0이상 1.0이하의 실수이다.    
그 원소들을 모두 더하게 되면 1.0이 나오게 되는데 이를 '확률'로 해석할 수 있다.  
$y_k = \frac{\exp(a_k)}{\sum_{i=1}^{n} \exp(a_i)}$

#### Cross Entropy Error
소프트 맥스의 출력을 받아 사용한다.  
$E = -\sum_{k} t_k \log y_k$  
이 때, $t_k$는 정답 레이블을 나타내고 정답 레이블은  
$t = [0, 0, 1]$  
원핫 벡터로 표현하여, 1이 아닌 값은 실질적으로 사용될 수 없게 된다.

#### Cross Entropy Error Mini Batch
$E = -\frac{1}{N} \sum_{n} \sum_{k} t_{nk} \log y_{nk}$  

$y_nk$는 신경망의 출력이고, $t_nk$는 정답 레이블이다.  

## 1.3.2 미분과 기울기
신경망의 학습 목표는 손실을 최소화하는 매개변수를 찾는 것이다.  
이때 중요한 것이 '미분'과 '기울기'이다.  
<br>
$L = f(x)$  

미분은 스칼라 값 뿐 아니라 다변수도 미분할 수 있다.  
<br>
$\frac{\partial L}{\partial \mathbf{x}} = \left[ \frac{\partial L}{\partial x_1}, \frac{\partial L}{\partial x_2}, \ldots, \frac{\partial L}{\partial x_n} \right]$  

각 원소에 대한 미분을 계산한 것이 기울기(gradient)이다.  

## 1.3.3 연쇄법칙
